In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/mmasnick/git/fhir-for-research/modules':
  os.chdir(r'/Users/mmasnick/git/fhir-for-research/modules')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap_external.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codecs.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/aliases.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/__init__.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/utf_8.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/abc.py": 1670352297.0, "/opt/homebrew/Cellar/python@3.10/3.10.9/Frameworks/Python.framew

In [2]:
# Load dependencies
from fhir_pyrate import Pirate
import pandas as pd

# Instantiate a Pirate object using the FHIR-PYrate library to query a test FHIR server
search = Pirate(
    auth=None,
    base_url="https://api.logicahealth.org/FHIRResearchSynthea/open/",
    print_request_url=True,
)

# Use the whimsically named `steal_bundles()` method to instantiate a search interaction
#
# For more information, see https://github.com/UMEssen/FHIR-PYrate/#pirate
bundles = search.steal_bundles(
    resource_type='Patient',
    request_params={
        "_count": 10 # Get 10 instances per page
    },
    num_pages=1 # Get 1 page (so a total of 10 instances)
)

# Execute the search and convert to a Pandas DataFrame
df = search.bundles_to_dataframe(bundles)

df.head(5)

https://api.logicahealth.org/FHIRResearchSynthea/open/Patient?_count=10


Query (Patient):   0%|          | 0/1 [00:00<?, ?it/s]

Query (Patient): 100%|██████████| 1/1 [00:00<00:00, 722.04it/s]

,resourceType,id,meta_versionId,meta_lastUpdated,meta_source,text_status,text_div,extension_0_url,extension_0_extension_0_url,extension_0_extension_0_valueCoding_system,...,address_0_country,maritalStatus_coding_0_system,maritalStatus_coding_0_code,maritalStatus_coding_0_display,maritalStatus_text,multipleBirthBoolean,communication_0_language_coding_0_system,communication_0_language_coding_0_code,communication_0_language_coding_0_display,communication_0_language_text
0,Patient,1,1,2023-04-06T20:52:11.000+00:00,#wQwWCylvgEiNKNbB,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",http://hl7.org/fhir/us/core/StructureDefinitio...,ombCategory,urn:oid:2.16.840.1.113883.6.238,...,US,http://terminology.hl7.org/CodeSystem/v3-Marit...,M,M,M,False,urn:ietf:bcp:47,en-US,English,English
1,Patient,180,1,2023-04-06T20:53:35.000+00:00,#N1sYXaxE6wGQilNA,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",http://hl7.org/fhir/us/core/StructureDefinitio...,ombCategory,urn:oid:2.16.840.1.113883.6.238,...,US,http://terminology.hl7.org/CodeSystem/v3-Marit...,S,Never Married,Never Married,False,urn:ietf:bcp:47,en-US,English,English
2,Patient,293,1,2023-04-06T20:56:31.000+00:00,#ZAAeVXzDXxeetcQS,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",http://hl7.org/fhir/us/core/StructureDefinitio...,ombCategory,urn:oid:2.16.840.1.113883.6.238,...,US,http://terminology.hl7.org/CodeSystem/v3-Marit...,S,S,S,False,urn:ietf:bcp:47,en-US,English,English
3,Patient,612,1,2023-04-06T20:57:01.000+00:00,#sTSlRxyopDL1X4fa,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",http://hl7.org/fhir/us/core/StructureDefinitio...,ombCategory,urn:oid:2.16.840.1.113883.6.238,...,US,http://terminology.hl7.org/CodeSystem/v3-Marit...,S,S,S,False,urn:ietf:bcp:47,en-US,English,English
4,Patient,931,1,2023-04-06T20:57:13.000+00:00,#ElqD7MaHoYuk6qSo,generated,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene...",http://hl7.org/fhir/us/core/StructureDefinitio...,ombCategory,urn:oid:2.16.840.1.113883.6.238,...,US,http://terminology.hl7.org/CodeSystem/v3-Marit...,S,S,S,False,urn:ietf:bcp:47,en-US,English,English


In [3]:
# Print the first row of the DataFrame vertically for easier reading.
pd.set_option('display.max_rows', 100)  # Show all rows
df.head(1).T

,0
resourceType,Patient
id,1
meta_versionId,1
meta_lastUpdated,2023-04-06T20:52:11.000+00:00
meta_source,#wQwWCylvgEiNKNbB
text_status,generated
text_div,"<div xmlns=""http://www.w3.org/1999/xhtml"">Gene..."
extension_0_url,http://hl7.org/fhir/us/core/StructureDefinitio...
extension_0_extension_0_url,ombCategory
extension_0_extension_0_valueCoding_system,urn:oid:2.16.840.1.113883.6.238


In [4]:
# Instantiate and perform the FHIR search interaction in a single function call
df = search.steal_bundles_to_dataframe(
    resource_type='Patient',
    request_params={
        "_count": 10  # Get 10 instances per page
    },
    num_pages=1,  # Get 1 page (so a total of 10 instances)
    fhir_paths=[
        ("id", "identifier[0].value"),
        ("gender", "gender"),
        ("date_of_birth", "birthDate"),
        ("marital_status", "maritalStatus.coding[0].code")
    ])
df

https://api.logicahealth.org/FHIRResearchSynthea/open/Patient?_count=10


Query & Build DF (Patient):   0%|          | 0/1 [00:00<?, ?it/s]

Query & Build DF (Patient): 100%|██████████| 1/1 [00:00<00:00, 198.33it/s]

,id,gender,date_of_birth,marital_status
0,439b24b4-6f25-4093-b101-47a39bd061ca,male,1955-10-09,M
1,99285aac-e5e3-4f5b-857d-f67271c97304,female,2017-12-02,S
2,2fa15bc7-8866-461a-9000-f739e425860a,male,1945-12-10,S
3,2fa15bc7-8866-461a-9000-f739e425860a,male,1945-12-10,S
4,2fa15bc7-8866-461a-9000-f739e425860a,male,1945-12-10,S
5,41166989-975d-4d17-b9de-17f94cb3eec1,male,1946-03-29,M
6,2b083021-e93f-4991-bf49-fd4f20060ef8,female,2002-10-24,S
7,29e51479-f742-4474-8f8e-d2607d5269f6,male,1999-12-12,S
8,262b819a-5193-404a-9787-b7f599358035,male,2002-04-15,S
9,aff8f143-2375-416f-901d-b0e4c73e3e58,male,1997-12-26,S


In [5]:
# Instantiate and perform the FHIR search interaction in a single function call
df = search.steal_bundles_to_dataframe(
    resource_type='Patient',
    request_params={
        "_count": 10  # Get 10 instances per page
    },
    num_pages=1,  # Get 1 page (so a total of 10 instances)
    fhir_paths=[
        ("id", "identifier[0].value"),
        ("identifiers", "identifier.value")
    ])
df

https://api.logicahealth.org/FHIRResearchSynthea/open/Patient?_count=10


Query & Build DF (Patient):   0%|          | 0/1 [00:00<?, ?it/s]

Query & Build DF (Patient): 100%|██████████| 1/1 [00:00<00:00, 684.45it/s]

,id,identifiers
0,439b24b4-6f25-4093-b101-47a39bd061ca,"[439b24b4-6f25-4093-b101-47a39bd061ca, 439b24b..."
1,99285aac-e5e3-4f5b-857d-f67271c97304,"[99285aac-e5e3-4f5b-857d-f67271c97304, 99285aa..."
2,2fa15bc7-8866-461a-9000-f739e425860a,"[2fa15bc7-8866-461a-9000-f739e425860a, 2fa15bc..."
3,2fa15bc7-8866-461a-9000-f739e425860a,"[2fa15bc7-8866-461a-9000-f739e425860a, 2fa15bc..."
4,2fa15bc7-8866-461a-9000-f739e425860a,"[2fa15bc7-8866-461a-9000-f739e425860a, 2fa15bc..."
5,41166989-975d-4d17-b9de-17f94cb3eec1,"[41166989-975d-4d17-b9de-17f94cb3eec1, 4116698..."
6,2b083021-e93f-4991-bf49-fd4f20060ef8,"[2b083021-e93f-4991-bf49-fd4f20060ef8, 2b08302..."
7,29e51479-f742-4474-8f8e-d2607d5269f6,"[29e51479-f742-4474-8f8e-d2607d5269f6, 29e5147..."
8,262b819a-5193-404a-9787-b7f599358035,"[262b819a-5193-404a-9787-b7f599358035, 262b819..."
9,aff8f143-2375-416f-901d-b0e4c73e3e58,"[aff8f143-2375-416f-901d-b0e4c73e3e58, aff8f14..."


In [6]:
df.join(
    pd.DataFrame(
        df.pop('identifiers').values.tolist()
    ).add_prefix('identifier_')
)

,id,identifier_0,identifier_1,identifier_2,identifier_3,identifier_4
0,439b24b4-6f25-4093-b101-47a39bd061ca,439b24b4-6f25-4093-b101-47a39bd061ca,439b24b4-6f25-4093-b101-47a39bd061ca,999-57-3355,S99925942,X42032818X
1,99285aac-e5e3-4f5b-857d-f67271c97304,99285aac-e5e3-4f5b-857d-f67271c97304,99285aac-e5e3-4f5b-857d-f67271c97304,999-62-8033,None,None
2,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,999-93-7537,S99948707,X14078167X
3,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,999-93-7537,S99948707,X14078167X
4,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,2fa15bc7-8866-461a-9000-f739e425860a,999-93-7537,S99948707,X14078167X
5,41166989-975d-4d17-b9de-17f94cb3eec1,41166989-975d-4d17-b9de-17f94cb3eec1,41166989-975d-4d17-b9de-17f94cb3eec1,999-17-8717,S99933732,X75257608X
6,2b083021-e93f-4991-bf49-fd4f20060ef8,2b083021-e93f-4991-bf49-fd4f20060ef8,2b083021-e93f-4991-bf49-fd4f20060ef8,999-83-6040,S99998967,None
7,29e51479-f742-4474-8f8e-d2607d5269f6,29e51479-f742-4474-8f8e-d2607d5269f6,29e51479-f742-4474-8f8e-d2607d5269f6,999-48-8328,S99955051,None
8,262b819a-5193-404a-9787-b7f599358035,262b819a-5193-404a-9787-b7f599358035,262b819a-5193-404a-9787-b7f599358035,999-19-7941,S99986287,None
9,aff8f143-2375-416f-901d-b0e4c73e3e58,aff8f143-2375-416f-901d-b0e4c73e3e58,aff8f143-2375-416f-901d-b0e4c73e3e58,999-35-9084,S99998334,X10377495X


In [7]:
# Instantiate and perform the FHIR search interaction in a single function call
dfs = search.steal_bundles_to_dataframe(
    resource_type='Patient',
    request_params={
        # Get instances of Observation where `Observation.patient` refers to a fetched Patient instance
        "_revinclude": "Observation:patient",

        "_count": 10  # Get 10 instances per page
    },
    num_pages=1,  # Get 1 page (so a total of 10 instances)
)

# `dfs` is a dictionary where the key is the FHIR resource type, and the value is the DataFrame
#
# Split these into separate variables for easy access:
df_patients = dfs['Patient']
df_observations = dfs['Observation']

# Look at the first row of the Observations DataFrame
df_observations.head(1).T

https://api.logicahealth.org/FHIRResearchSynthea/open/Patient?_count=10&_revinclude=Observation:patient


Query & Build DF (Patient):   0%|          | 0/1 [00:00<?, ?it/s]

Query & Build DF (Patient): 100%|██████████| 1/1 [00:00<00:00, 43.19it/s]

,0
resourceType,Observation
id,2048
meta_versionId,1
meta_lastUpdated,2023-04-06T20:58:59.000+00:00
meta_source,#aCMN9zUZkrDZoyGR
status,final
category_0_coding_0_system,http://terminology.hl7.org/CodeSystem/observat...
category_0_coding_0_code,vital-signs
category_0_coding_0_display,vital-signs
code_coding_0_system,http://loinc.org


In [8]:
df_observations2 = search.trade_rows_for_dataframe(
    df_patients,
    resource_type="Observation",

    # Limit to 10 instances of Observation per patient
    request_params={
        "_count": "10"
    },
    num_pages=1,

    # Load Observations where `Observation.subject` references the instance of Patient
    # identified by `id` in the `df_patients` DataFrame
    df_constraints={"subject": "id"},
    fhir_paths=[
      ("observation_id", "id"),
      ("patient", "subject.reference"),
      ("status", "status"),
      ("code", "code.coding[0].code"),
      ("code_display", "code.coding[0].display"),
      ("value", "valueQuantity.value"),
      ("value_units", "valueQuantity.unit"),
      ("datetime", "effectiveDateTime")
    ]
)

# Look at the first row of the Observations DataFrame
df_observations2.head(15)

Query & Build DF (Observation):   0%|          | 0/10 [00:00<?, ?it/s]

Query & Build DF (Observation):   0%|          | 0/10 [00:00<?, ?it/s]

Query & Build DF (Observation):  10%|█         | 1/10 [00:00<00:02,  3.37it/s]

https://api.logicahealth.org/FHIRResearchSynthea/open/Observation?_count=10&subject=1


Query & Build DF (Observation):  10%|█         | 1/10 [00:00<00:02,  3.37it/s]

...

Query & Build DF (Observation):  90%|█████████ | 9/10 [00:03<00:00,  2.96it/s]

Query & Build DF (Observation): 100%|██████████| 10/10 [00:03<00:00,  3.09it/s]

Query & Build DF (Observation): 100%|██████████| 10/10 [00:03<00:00,  3.12it/s]

https://api.logicahealth.org/FHIRResearchSynthea/open/Observation?_count=10&subject=2370


,observation_id,patient,status,code,code_display,datetime,value,value_units,id
0,122,Patient/1,final,72166-2,Tobacco smoking status NHIS,2015-12-06T14:48:37-05:00,NaN,NaN,1
1,121,Patient/1,final,55284-4,Blood Pressure,2015-12-06T14:48:37-05:00,NaN,NaN,1
2,61,Patient/1,final,72166-2,Tobacco smoking status NHIS,2011-11-13T14:48:37-05:00,NaN,NaN,1
3,120,Patient/1,final,39156-5,Body Mass Index,2015-12-06T14:48:37-05:00,25.783578,kg/m2,1
4,60,Patient/1,final,2085-9,High Density Lipoprotein Cholesterol,2011-11-13T14:48:37-05:00,79.213320,mg/dL,1
5,69,Patient/1,final,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2012-11-18T14:48:37-05:00,0.845428,{score},1
6,68,Patient/1,final,8302-2,Body Height,2012-11-18T14:48:37-05:00,190.900092,cm,1
7,70,Patient/1,final,29463-7,Body Weight,2012-11-18T14:48:37-05:00,95.644696,kg,1
8,110,Patient/1,final,72166-2,Tobacco smoking status NHIS,2014-11-30T14:48:37-05:00,NaN,NaN,1
9,72,Patient/1,final,55284-4,Blood Pressure,2012-11-18T14:48:37-05:00,NaN,NaN,1


In [9]:
df_observations2 = search.trade_rows_for_dataframe(
    df_patients,
    resource_type="Observation",

    # Limit to 10 instances of Observation per patient
    request_params={
        "_count": "10",
        "code": "http://loinc.org|72166-2"
    },
    num_pages=1,

    # Load Observations where `Observation.subject` references the instance of Patient
    # identified by `id` in the `df_patients` DataFrame
    df_constraints={"subject": "id"},
    fhir_paths=[
      ("observation_id", "id"),
      ("patient", "subject.reference"),
      ("status", "status"),
      ("code", "code.coding[0].code"),
      ("code_display", "code.coding[0].display"),
      ("value", "valueCodeableConcept.coding[0].code"),
      ("value_display", "valueCodeableConcept.coding[0].display"),
      ("datetime", "effectiveDateTime")
    ]
)

# Look at the first row of the Observations DataFrame
df_observations2.head(15)

Query & Build DF (Observation):   0%|          | 0/10 [00:00<?, ?it/s]

Query & Build DF (Observation):   0%|          | 0/10 [00:00<?, ?it/s]

Query & Build DF (Observation):  10%|█         | 1/10 [00:00<00:02,  3.81it/s]

https://api.logicahealth.org/FHIRResearchSynthea/open/Observation?_count=10&code=http://loinc.org|72166-2&subject=1


Query & Build DF (Observation):  10%|█         | 1/10 [00:00<00:02,  3.81it/s]

...

Query & Build DF (Observation):  90%|█████████ | 9/10 [00:02<00:00,  3.67it/s]

Query & Build DF (Observation): 100%|██████████| 10/10 [00:02<00:00,  3.63it/s]

Query & Build DF (Observation): 100%|██████████| 10/10 [00:02<00:00,  3.77it/s]

https://api.logicahealth.org/FHIRResearchSynthea/open/Observation?_count=10&code=http://loinc.org|72166-2&subject=2370


,observation_id,patient,status,code,code_display,value,value_display,datetime,id
0,122,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2015-12-06T14:48:37-05:00,1
1,61,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2011-11-13T14:48:37-05:00,1
2,110,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2014-11-30T14:48:37-05:00,1
3,84,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2012-11-18T14:48:37-05:00,1
4,95,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2013-11-24T14:48:37-05:00,1
5,136,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2016-12-11T14:48:37-05:00,1
6,163,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2017-12-17T14:48:37-05:00,1
7,20,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2009-11-01T14:48:37-05:00,1
8,42,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2010-11-07T14:48:37-05:00,1
9,176,Patient/1,final,72166-2,Tobacco smoking status NHIS,266919005,Never smoker,2018-12-23T14:48:37-05:00,1
